# CNF benchmarks comparison
tested benchmarks: uf20-01 -- uf20-1000

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc

rc('font',**{'family':'serif','serif':['Helvetica']})
rc('text', usetex=True)

In [8]:
dimacs_nodecount_csv = "./csv/dimacs20-nc.csv"
dimacs_boxcount_csv = "./csv/dimacs20-boxes.csv"
dimacs_bc_output = "./figures/dimacs-abdd-box-counts.pdf"
dimacs_nc_output = "./figures/dimacs-abdd-vs-all.pdf"

In [9]:
df = pd.read_csv(dimacs_nodecount_csv)
df.rename(columns={'esr': 'esrbdd'}, inplace=True)

In [10]:
def getDimacsStats(df: pd.DataFrame, order):
    result = {
        "name": [],
        "sum": [],
        "max": [],
        "avg": []
    }
    for model in order:
        result['name'].append(model)
        result['sum'].append(df[model].sum())
        result['max'].append(df[model].max())
        result['avg'].append(df[model].mean())

    return pd.DataFrame(result).transpose()

stat_df = getDimacsStats(df, ['bdd', 'zbdd', 'tbdd', 'czdd', 'cbdd', 'esrbdd', 'abdd'])
print(stat_df)
stat_df.to_latex('test')

def computeAvgReduction(data: pd.DataFrame, reference: str, target: str):
    reductions = []
    for i in range(len(data[reference])):
        ref = data[reference][i]
        val = data[target][i]
        res = (ref - val) / ref
        reductions.append(res)
    return round((sum(reductions) / len(reductions)) * 100, ndigits=3)

print('(bdd-abdd) / bdd = (%)', computeAvgReduction(df, 'bdd', 'abdd'))
print('(zbdd-abdd) / zbdd = (%)', computeAvgReduction(df, 'zbdd', 'abdd'))
print('(tbdd-abdd) / tbdd = (%)', computeAvgReduction(df, 'tbdd', 'abdd'))
print('(esrbdd-abdd) / esrbdd = (%)', computeAvgReduction(df, 'esrbdd', 'abdd'))
print('(czdd-abdd) / czdd = (%)', computeAvgReduction(df, 'czdd', 'abdd'))
print('(cbdd-abdd) / cbdd = (%)', computeAvgReduction(df, 'cbdd', 'abdd'))

           0       1       2       3       4       5       6
name     bdd    zbdd    tbdd    czdd    cbdd  esrbdd    abdd
sum    45517   28591   45041   27845   42002   27676   26713
max      287     216     287     206     242     177     160
avg   45.517  28.591  45.041  27.845  42.002  27.676  26.713
(bdd-abdd) / bdd = (%) 41.282
(zbdd-abdd) / zbdd = (%) 0.426
(tbdd-abdd) / tbdd = (%) 40.452
(esrbdd-abdd) / esrbdd = (%) 2.19
(czdd-abdd) / czdd = (%) -1.675
(cbdd-abdd) / cbdd = (%) 36.962


In [11]:
box_df = pd.read_csv(dimacs_boxcount_csv)
box_df = box_df[~box_df['benchmark'].str.startswith('C499')]
box_df = box_df.reset_index(drop=True)
box_df = box_df.drop(columns=['benchmark', 'norm', 'full'])
box_df.loc['Total'] = box_df.sum()
all_box_usage_total = box_df.iloc[-1].sum()

box_df.columns = ['$\mathtt{X}$', '$\mathtt{L}_\mathbf{0}$', '$\mathtt{L}_\mathbf{1}$', '$\mathtt{L}_\oplus$', '$\mathtt{H}_\mathbf{0}$', '$\mathtt{H}_\mathbf{1}$', '$\mathtt{H}_\oplus$']

box_avg_df = pd.DataFrame({
    'box' : box_df.columns,
    'average' : box_df.loc['Total'] / all_box_usage_total,
})
def addlabels(x,y):
    for i in range(len(x)):
        height = y[i] - 10 if y[i] * 100 + 1 > 10 else 0
        plt.text(i, y[i] * 100 + 1, round(y[i] * 100, ndigits=2), ha = 'center')

plt.bar(box_avg_df['box'], box_avg_df['average'] * 100, bottom=0, color='#6c8ebf')
plt.xlabel('Box used')
plt.ylabel('\% of usage in the benchmarks')
plt.ylim(0, 100)
addlabels(box_avg_df['box'], box_avg_df['average'])
plt.savefig(dimacs_bc_output, format='pdf', dpi=300, facecolor='white', bbox_inches='tight', pad_inches=0.05)
plt.clf()


accent_color = '#6c8ebf' # poster color - light blue
title_color = '#10263b' # poster color - navy blue 

<Figure size 432x288 with 0 Axes>

In [12]:
df.rename(columns={'esr': 'esrbdd'}, inplace=True)

color_1 = "#3dd9c1"
color_2 = "#3d5cd9"
color_3 = "#3dd954"
color_4 = "#e88700"
color_5 = "#d9453d"
color_6 = "#d93dd1"

fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(df['zbdd'],   df['abdd'], c=color_2, marker='3', s=50, linewidths=1, label='ZBDD')
ax1.scatter(df['cbdd'],   df['abdd'], c=color_3, marker='+', s=50, linewidths=1, label='CBDD')
ax1.scatter(df['czdd'],   df['abdd'], c=color_4, marker='1', s=50, linewidths=1, label='CZDD')
ax1.scatter(df['tbdd'],   df['abdd'], c=color_5, marker='2', s=50, linewidths=1, label='TBDD')
ax1.scatter(df['esrbdd'], df['abdd'], c=color_6, marker='x', s=50, linewidths=1, label='ESRBDD')
ax1.scatter(df['bdd'],    df['abdd'], c=color_1, marker='4', s=50, linewidths=1, label='BDD')
plt.xlabel(f'Other BDD model node count')
plt.ylabel(f'ABDD node count')
ax1.legend()
plt.xlim((0, 300))
plt.ylim((0, 300))
plt.plot([0, 1], [0, 1], transform=plt.gca().transAxes, ls='--', c='black')

# plt.show()
plt.savefig(dimacs_nc_output, format='pdf', dpi=300, facecolor='white',
            edgecolor='r', bbox_inches='tight', pad_inches=0.1)
plt.clf()


<Figure size 432x288 with 0 Axes>